In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('gan')):
    os.mkdir('saved_models_{}'.format('gan'))
f = open('saved_models_{}/log_collapse1.txt'.format('gan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam #optimizer of keras

import matplotlib.pyplot as plt
from PIL import Image
import sys

import numpy as np
import os
class GAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels) #shape of image
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5) #optimizer of gan

        # Build and compile the discriminator,only to keras
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))  #Input():用来实例化一个keras张量
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5

        # X_train = np.expand_dims(X_train, axis=3)  #expand_dims用于扩充数组形状
        print(np.shape(X_train))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Train the generator (to have the discriminator label samples as valid)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))

                if global_step % sample_interval == 0:
                    self.mode_drop(epoch, global_step)
                    

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')

if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 1,704,961
Trainable params: 1,704,961
Non-trainable params: 0
_________________________________________________________________


/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.738315, acc.: 48.44%] [G loss: 0.658707]
epoch:0 step:2 [D loss: 0.595730, acc.: 60.94%] [G loss: 0.624397]
epoch:0 step:3 [D loss: 0.567147, acc.: 59.38%] [G loss: 0.672657]
epoch:0 step:4 [D loss: 0.499619, acc.: 67.97%] [G loss: 0.705538]
epoch:0 step:5 [D loss: 0.519506, acc.: 63.28%] [G loss: 0.749025]
epoch:0 step:6 [D loss: 0.484215, acc.: 70.31%] [G loss: 0.814351]
epoch:0 step:7 [D loss: 0.475841, acc.: 72.66%] [G loss: 0.893869]
epoch:0 step:8 [D loss: 0.517761, acc.: 65.62%] [G loss: 0.938398]
epoch:0 step:9 [D loss: 0.470900, acc.: 75.00%] [G loss: 0.961686]
epoch:0 step:10 [D loss: 0.466170, acc.: 75.00%] [G loss: 1.005195]
epoch:0 step:11 [D loss: 0.452090, acc.: 75.78%] [G loss: 1.004033]
epoch:0 step:12 [D loss: 0.409472, acc.: 81.25%] [G loss: 1.076959]
epoch:0 step:13 [D loss: 0.432678, acc.: 78.12%] [G loss: 1.044729]
epoch:0 step:14 [D loss: 0.438130, acc.: 78.91%] [G loss: 1.126150]
epoch:0 step:15 [D loss: 0.374202, acc.: 82.81%] [G loss:

epoch:0 step:125 [D loss: 0.604209, acc.: 69.53%] [G loss: 1.549466]
epoch:0 step:126 [D loss: 0.657025, acc.: 64.84%] [G loss: 1.446825]
epoch:0 step:127 [D loss: 0.604687, acc.: 73.44%] [G loss: 1.453085]
epoch:0 step:128 [D loss: 0.658838, acc.: 57.81%] [G loss: 1.603979]
epoch:0 step:129 [D loss: 0.716307, acc.: 54.69%] [G loss: 1.458737]
epoch:0 step:130 [D loss: 0.674877, acc.: 58.59%] [G loss: 1.382178]
epoch:0 step:131 [D loss: 0.766708, acc.: 54.69%] [G loss: 1.525617]
epoch:0 step:132 [D loss: 0.605930, acc.: 66.41%] [G loss: 1.776689]
epoch:0 step:133 [D loss: 0.686491, acc.: 59.38%] [G loss: 1.569793]
epoch:0 step:134 [D loss: 0.681302, acc.: 59.38%] [G loss: 1.380345]
epoch:0 step:135 [D loss: 0.704147, acc.: 58.59%] [G loss: 1.415540]
epoch:0 step:136 [D loss: 0.632356, acc.: 64.06%] [G loss: 1.437289]
epoch:0 step:137 [D loss: 0.644284, acc.: 68.75%] [G loss: 1.547157]
epoch:0 step:138 [D loss: 0.641474, acc.: 66.41%] [G loss: 1.392242]
epoch:0 step:139 [D loss: 0.644494

...

epoch:49 step:38525 [D loss: 0.313283, acc.: 85.16%] [G loss: 3.105251]
epoch:49 step:38526 [D loss: 0.307196, acc.: 85.16%] [G loss: 3.486682]
epoch:49 step:38527 [D loss: 0.278784, acc.: 91.41%] [G loss: 3.104979]
epoch:49 step:38528 [D loss: 0.278955, acc.: 85.94%] [G loss: 3.016917]
epoch:49 step:38529 [D loss: 0.396169, acc.: 81.25%] [G loss: 3.229456]
epoch:49 step:38530 [D loss: 0.398672, acc.: 84.38%] [G loss: 2.872988]
epoch:49 step:38531 [D loss: 0.364569, acc.: 87.50%] [G loss: 3.219414]
epoch:49 step:38532 [D loss: 0.370527, acc.: 87.50%] [G loss: 4.080089]
epoch:49 step:38533 [D loss: 0.302106, acc.: 85.16%] [G loss: 3.599767]
epoch:49 step:38534 [D loss: 0.363183, acc.: 86.72%] [G loss: 2.800434]
epoch:49 step:38535 [D loss: 0.290192, acc.: 87.50%] [G loss: 3.623133]
epoch:49 step:38536 [D loss: 0.271943, acc.: 89.84%] [G loss: 3.469874]
epoch:49 step:38537 [D loss: 0.233527, acc.: 90.62%] [G loss: 3.217194]
epoch:49 step:38538 [D loss: 0.212833, acc.: 89.06%] [G loss: 3.

epoch:49 step:38644 [D loss: 0.217739, acc.: 92.19%] [G loss: 5.162095]
epoch:49 step:38645 [D loss: 0.263408, acc.: 89.06%] [G loss: 4.751713]
epoch:49 step:38646 [D loss: 0.186407, acc.: 89.84%] [G loss: 4.530573]
epoch:49 step:38647 [D loss: 0.231670, acc.: 91.41%] [G loss: 3.896189]
epoch:49 step:38648 [D loss: 0.310189, acc.: 84.38%] [G loss: 2.448632]
epoch:49 step:38649 [D loss: 0.282807, acc.: 88.28%] [G loss: 3.673183]
epoch:49 step:38650 [D loss: 0.221377, acc.: 89.84%] [G loss: 3.249962]
epoch:49 step:38651 [D loss: 0.391969, acc.: 80.47%] [G loss: 2.879026]
epoch:49 step:38652 [D loss: 0.288166, acc.: 89.84%] [G loss: 2.878206]
epoch:49 step:38653 [D loss: 0.270741, acc.: 86.72%] [G loss: 2.855588]
epoch:49 step:38654 [D loss: 0.206122, acc.: 91.41%] [G loss: 2.667804]
epoch:49 step:38655 [D loss: 0.424846, acc.: 78.91%] [G loss: 2.730810]
epoch:49 step:38656 [D loss: 0.316129, acc.: 85.16%] [G loss: 2.774994]
epoch:49 step:38657 [D loss: 0.314251, acc.: 85.16%] [G loss: 2.

epoch:49 step:38758 [D loss: 0.247421, acc.: 88.28%] [G loss: 5.301094]
epoch:49 step:38759 [D loss: 0.329545, acc.: 84.38%] [G loss: 3.011705]
epoch:49 step:38760 [D loss: 0.284792, acc.: 85.94%] [G loss: 4.915022]
epoch:49 step:38761 [D loss: 0.302914, acc.: 89.84%] [G loss: 2.543296]
epoch:49 step:38762 [D loss: 0.197939, acc.: 94.53%] [G loss: 3.052564]
epoch:49 step:38763 [D loss: 0.339851, acc.: 85.16%] [G loss: 3.810809]
epoch:49 step:38764 [D loss: 0.302139, acc.: 89.06%] [G loss: 2.759498]
epoch:49 step:38765 [D loss: 0.275756, acc.: 89.06%] [G loss: 3.300409]
epoch:49 step:38766 [D loss: 0.220611, acc.: 93.75%] [G loss: 4.055134]
epoch:49 step:38767 [D loss: 0.302296, acc.: 87.50%] [G loss: 3.061065]
epoch:49 step:38768 [D loss: 0.235085, acc.: 90.62%] [G loss: 3.187948]
epoch:49 step:38769 [D loss: 0.327170, acc.: 87.50%] [G loss: 2.957452]
epoch:49 step:38770 [D loss: 0.272673, acc.: 86.72%] [G loss: 3.341798]
epoch:49 step:38771 [D loss: 0.305643, acc.: 85.16%] [G loss: 3.

epoch:49 step:38878 [D loss: 0.332676, acc.: 85.16%] [G loss: 3.262698]
epoch:49 step:38879 [D loss: 0.392115, acc.: 81.25%] [G loss: 2.654981]
epoch:49 step:38880 [D loss: 0.356029, acc.: 82.03%] [G loss: 3.921971]
epoch:49 step:38881 [D loss: 0.284003, acc.: 89.84%] [G loss: 2.627433]
epoch:49 step:38882 [D loss: 0.309366, acc.: 86.72%] [G loss: 2.764746]
epoch:49 step:38883 [D loss: 0.192204, acc.: 93.75%] [G loss: 3.067501]
epoch:49 step:38884 [D loss: 0.333305, acc.: 85.94%] [G loss: 2.862129]
epoch:49 step:38885 [D loss: 0.247539, acc.: 88.28%] [G loss: 3.412580]
epoch:49 step:38886 [D loss: 0.386051, acc.: 84.38%] [G loss: 3.462824]
epoch:49 step:38887 [D loss: 0.320284, acc.: 84.38%] [G loss: 3.896941]
epoch:49 step:38888 [D loss: 0.249221, acc.: 91.41%] [G loss: 3.090870]
epoch:49 step:38889 [D loss: 0.274817, acc.: 89.06%] [G loss: 3.858710]
epoch:49 step:38890 [D loss: 0.181114, acc.: 92.97%] [G loss: 3.655122]
epoch:49 step:38891 [D loss: 0.257232, acc.: 90.62%] [G loss: 3.

epoch:49 step:38994 [D loss: 0.181437, acc.: 91.41%] [G loss: 3.871508]
epoch:49 step:38995 [D loss: 0.298179, acc.: 85.94%] [G loss: 2.508578]
epoch:49 step:38996 [D loss: 0.259960, acc.: 88.28%] [G loss: 3.618102]
epoch:49 step:38997 [D loss: 0.299381, acc.: 84.38%] [G loss: 3.600430]
epoch:49 step:38998 [D loss: 0.252123, acc.: 90.62%] [G loss: 3.261596]
epoch:49 step:38999 [D loss: 0.264440, acc.: 87.50%] [G loss: 3.501021]
epoch:49 step:39000 [D loss: 0.311698, acc.: 86.72%] [G loss: 2.892127]
epoch:49 step:39001 [D loss: 0.315091, acc.: 85.94%] [G loss: 4.002990]
epoch:49 step:39002 [D loss: 0.275611, acc.: 88.28%] [G loss: 3.520416]
epoch:49 step:39003 [D loss: 0.312554, acc.: 85.94%] [G loss: 3.695688]
epoch:49 step:39004 [D loss: 0.242845, acc.: 92.19%] [G loss: 2.795382]
epoch:49 step:39005 [D loss: 0.252721, acc.: 85.94%] [G loss: 3.677499]
epoch:49 step:39006 [D loss: 0.336928, acc.: 87.50%] [G loss: 2.727203]
epoch:49 step:39007 [D loss: 0.385652, acc.: 77.34%] [G loss: 3.